In [84]:
import pandas as pd
import numpy as np
from utils import fuel_algo

In [68]:
def fuel_algo2(df, displacement):
    sec_hour = 3600
    air_to_fuel = 14.08
    fuel_density = 820

    rate = df['Fuel Rate[L/hr]']
    maf = df['MAF[g/sec]']
    out = np.array(rate)
    
    idx = rate.isna()
    out[idx] = maf[idx] * air_to_fuel
    return out

In [90]:
trip = pd.read_csv('ICE_trip7636.csv')
x = np.array(trip['Fuel Rate[L/hr]'])
x = fuel_algo(trip, '1.5L')
print(np.mean(x))


2.953216464619631
